In [5]:
import pandas as pd

import string 
df_reviews = pd.read_csv('IMDB_Dataset.csv')

df_reviews_teste = df_reviews.sample(20)

df_filtrado_positivo = df_reviews_teste[df_reviews_teste['sentiment']=='positive']
df_filtrado_negativo = df_reviews_teste[df_reviews_teste['sentiment']=='negative']

dict_palavras_pos={}
for review in df_filtrado_positivo['review']:
    review=review.replace('<br />', " ")
    filtro_caractranslator= str.maketrans('','', string.punctuation) 
    review = review.translate(translator) in set(lista_reviews_semi_filtrada):
        if palavra in dict_palavras_pos:
            dict_palavras_pos[palavra]+=1
        else:
            dict_palavras_pos[palavra]=1

dict_palavras_neg={}
for review in df_filtrado_negativo['review']:
    review=review.replace('<br />', " ")
    filtro_caracter=[',','(', '/', '(',')',';',':','.','*','"',"'",'[',']', '-', '?', '!', '=']
    for caracter in review:
        if caracter in filtro_caracter:
            review=review.replace(caracter,'').lower()
    lista_reviews_semi_filtrada = review.split()
    for palavra in set(lista_reviews_semi_filtrada):
        if palavra in dict_palavras_neg:
            dict_palavras_neg[palavra]+=1
        else:
            dict_palavras_neg[palavra]=1



In [6]:
#calcular P(palavra|classe)

#positivo
quant_review_pos = len(df_filtrado_positivo)
dict_prob_dado_pos = {}

for palavra, quant in dict_palavras_pos.items():
    prob = quant/quant_review_pos
    dict_prob_dado_pos[palavra] = prob



#negativo
quant_review_neg = len(df_filtrado_negativo)
dict_prob_dado_neg = {}

for palavra, quant in dict_palavras_neg.items():
    prob = quant/quant_review_neg
    dict_prob_dado_neg[palavra] = prob

#calcular P(nao palavra|classe)

#positvo
dict_prob_n_palavra_dado_pos = {}
for palavra, prob in dict_prob_dado_pos.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_pos[palavra] = prob_n_palavra

#negativo
dict_prob_n_palavra_dado_neg = {}
for palavra, prob in dict_prob_dado_neg.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_neg[palavra] = prob_n_palavra

In [8]:
#calcular P(palavra)

palavras_pos=len(df_filtrado_positivo)
palavras_neg=len(df_filtrado_negativo)
total_palavras=palavras_pos+palavras_neg

print(dict_palavras_neg)

probalidade_palavras={}

for palavra in dict_palavras_pos:
    palavra_valor = 0
    try:
        palavra_valor += dict_palavras_neg[palavra]
    except:
        pass
    palavra_valor += dict_palavras_pos[palavra]
    p_palavra=palavra_valor/total_palavras
    probalidade_palavras[palavra]=p_palavra


for palavra in dict_palavras_neg:
    if not(palavra in probalidade_palavras):
        palavra_valor = 0
        try:
            palavra_valor += dict_palavras_pos[palavra]
        except:
            pass
        palavra_valor += dict_palavras_neg[palavra]
        p_palavra=palavra_valor/total_palavras
        if palavra in probalidade_palavras:
            probalidade_palavras[palavra]+=p_palavra
        else:
            probalidade_palavras[palavra]=p_palavra


print(probalidade_palavras)


{'make': 2, 'more': 6, 'seems': 2, 'laugh': 1, 'of': 10, 'word': 1, 'no': 5, 'for': 7, 'lots': 1, 'movies': 5, 'horton': 1, 'this': 9, 'darkness': 1, 'crap': 1, 'scenes': 2, 'bad': 5, 'hope': 2, 'such': 2, 'these': 4, 'life': 2, 'two': 5, 'camera': 2, 'please': 3, 'any': 7, 'if': 7, 'movie': 9, 'i': 10, 'them': 3, 'like': 7, 'which': 5, 'seen': 3, 'other': 2, 'wont': 1, 'bankrupts': 1, 'simply': 2, 'use': 3, 'nothing': 6, 'worst': 3, 'terrible': 1, 'but': 11, 'hubbel': 1, 'a': 10, 'illogical': 1, 'zombies': 1, 'boring': 2, 'be': 6, 'we': 4, 'vs': 1, 'can': 4, 'days': 3, 'many': 4, 'you': 9, 'the': 11, 'even': 5, 'conversation': 1, 'stop': 3, 'watch': 2, 'hand': 1, 'ive': 1, 'because': 3, 'shannon': 1, 'there': 4, '2': 3, 'directors': 1, 'and': 11, 'making': 3, 'your': 2, 'they': 5, 'in': 10, 'to': 10, 'on': 6, 'friends': 4, 'one': 10, 'dont': 6, 'minute': 1, 'ever': 2, 'are': 6, 'waste': 1, 'vampire': 1, 'jason': 1, '80%': 1, 'some': 2, 'or': 3, 'with': 8, 'is': 9, 'guys': 1, 'created'